In [4]:
from dotenv import load_dotenv
load_dotenv()

import fitz
import json
import os
from groq import Groq
from neo4j import GraphDatabase


In [5]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text


In [6]:
from groq import Groq
import os, json

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def analyze_contract(text):
    prompt = f"""
You are a legal risk analysis AI.

STRICT RULES:
- Return ONLY valid JSON
- Do NOT include markdown, comments, or extra text
- If something is missing, use empty string ""

For EACH clause:
1. Identify risk level
2. Explain WHY it is risky
3. Identify obligations
4. Identify liabilities
5. Identify missing safeguards (if any)
6. Provide a clear plain-English AI summary

JSON FORMAT (exact):

{{
  "title": "",
  "parties": [],
  "dates": [],
  "governing_law": "",
  "clauses": [
    {{
      "clause_name": "",
      "summary": "",
      "risk_level": "LOW | MEDIUM | HIGH",
      "risk_reason": "",
      "obligations": "",
      "liabilities": "",
      "missing_safeguards": "",
      "ai_summary": ""
    }}
  ]
}}

Contract text:
{text[:9000]}
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=2000
    )

    raw_output = response.choices[0].message.content.strip()

    # 🛡️ SAFETY: handle invalid JSON from LLM
    try:
        return json.loads(raw_output)
    except json.JSONDecodeError:
        print("❌ Invalid JSON returned by AI")
        print("----- RAW OUTPUT START -----")
        print(raw_output)
        print("----- RAW OUTPUT END -----")
        raise


In [7]:
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)


In [8]:
def store_graph(data):
    with driver.session() as session:

        # Contract
        session.run("""
            MERGE (c:Contract {title:$title})
            SET c.governing_law = $law
        """, title=data["title"], law=data["governing_law"])

        # Parties
        for party in data["parties"]:
            session.run("""
                MERGE (o:Organization {name:$name})
                WITH o
                MATCH (c:Contract {title:$title})
                MERGE (o)-[:IS_PARTY_TO]->(c)
            """, name=party, title=data["title"])

        # Dates
        for d in data["dates"]:
            session.run("""
                MERGE (d:ImportantDate {value:$date})
                WITH d
                MATCH (c:Contract {title:$title})
                MERGE (c)-[:HAS_DATE]->(d)
            """, date=d, title=data["title"])

        # Clauses + rich risk analysis
        for cl in data["clauses"]:
            session.run("""
                MATCH (c:Contract {title:$title})

                CREATE (clause:Clause {
                    name: $name,
                    summary: $summary
                })

                CREATE (risk:Risk {level: $risk_level})
                CREATE (reason:RiskReason {text: $risk_reason})
                CREATE (ob:Obligation {text: $obligations})
                CREATE (liab:Liability {text: $liabilities})
                CREATE (miss:MissingSafeguard {text: $missing})
                CREATE (ai:AISummary {text: $ai_summary})

                CREATE (c)-[:HAS_CLAUSE]->(clause)
                CREATE (clause)-[:HAS_RISK]->(risk)
                CREATE (clause)-[:HAS_REASON]->(reason)
                CREATE (clause)-[:HAS_OBLIGATION]->(ob)
                CREATE (clause)-[:HAS_LIABILITY]->(liab)
                CREATE (clause)-[:HAS_MISSING_SAFEGUARD]->(miss)
                CREATE (clause)-[:HAS_AI_SUMMARY]->(ai)
            """,
            title=data["title"],
            name=cl["clause_name"],
            summary=cl["summary"],
            risk_level=cl["risk_level"],
            risk_reason=cl["risk_reason"],
            obligations=cl["obligations"],
            liabilities=cl["liabilities"],
            missing=cl["missing_safeguards"],
            ai_summary=cl["ai_summary"]
            )


In [10]:
def print_contract_summary(data):
    print("\n" + "="*60)
    print("📄 CONTRACT SUMMARY (AI GENERATED)")
    print("="*60)

    print(f"\n🔹 Title: {data.get('title','')}")
    print(f"⚖️ Governing Law: {data.get('governing_law','')}")
    
    print("\n👥 Parties:")
    for p in data.get("parties", []):
        print(f"  - {p}")

    print("\n📌 CLAUSE ANALYSIS:\n")

    for i, cl in enumerate(data.get("clauses", []), start=1):
        print(f"Clause {i}: {cl.get('clause_name','')}")
        print(f"  📄 Summary          : {cl.get('summary','')}")
        print(f"  ⚠️ Risk Level       : {cl.get('risk_level','')}")
        print(f"  ❓ Risk Reason      : {cl.get('risk_reason','')}")
        print(f"  📌 Obligations     : {cl.get('obligations','')}")
        print(f"  ⚖️ Liabilities     : {cl.get('liabilities','')}")
        print(f"  🚨 Missing Safeguard: {cl.get('missing_safeguards','')}")
        print(f"  🧠 AI Explanation   : {cl.get('ai_summary','')}")
        print("-"*60)


In [12]:
import os

def process_multiple_contracts(folder_path="contracts"):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]

    print(f"\n📂 Total contracts found: {len(pdf_files)}\n")

    for i, pdf in enumerate(pdf_files, start=1):
        print("\n" + "="*70)
        print(f"📄 Contract {i}: {pdf}")
        print("="*70)

        pdf_path = os.path.join(folder_path, pdf)

        text = extract_text_from_pdf(pdf_path)
        data = analyze_contract(text)

        # 🔹 Make title unique per PDF (IMPORTANT)
        data["title"] = f"{data['title']} ({pdf})"

        # 🔹 Show summary in terminal
        print_contract_summary(data)

        # 🔹 Store in Neo4j
        store_graph(data)

        print(f"\n✅ Finished processing {pdf}")


In [13]:
process_multiple_contracts("contracts")



📂 Total contracts found: 2


📄 Contract 1: contract2.pdf

📄 CONTRACT SUMMARY (AI GENERATED)

🔹 Title: Non-Disclosure Agreement (contract2.pdf)
⚖️ Governing Law: India

👥 Parties:
  - TechNova Solutions Pvt Ltd
  - Alpha Innovations LLP

📌 CLAUSE ANALYSIS:

Clause 1: Purpose
  📄 Summary          : Receiving Party agrees to receive confidential information for evaluating business collaboration.
  ⚠️ Risk Level       : LOW
  ❓ Risk Reason      : Low risk as the purpose is clearly defined and limited.
  📌 Obligations     : Receiving Party must use confidential information solely for stated purpose.
  ⚖️ Liabilities     : Receiving Party is liable for unauthorized disclosure.
  🚨 Missing Safeguard: 
  🧠 AI Explanation   : This clause defines the purpose of receiving confidential information, which is low-risk as it's limited to evaluating business collaboration.
------------------------------------------------------------
Clause 2: Definition of Confidential Information
  📄 Summary        